In [ ]:
cd /content/drive/MyDrive/Flask_model

/content/drive/MyDrive/Flask_model


In [ ]:
import os

if not os.path.exists('templates'):
  os.makedirs('templates')
if not os.path.exists('static'):
  os.makedirs('static')

In [ ]:
!pip install pyngrok==4.1.1
!ngrok authtoken '2KO3FCQwOe2Zm7VMe8aoAh4NogP_GZ9XTE9K1DqLNjYLZJbt'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15982 sha256=2d39a02c73721c780349f38c71765981c3bbfd575bae62c135637ad0392feb06
  Stored in directory: /root/.cache/pip/wheels/5e/0a/51/8cb053ccd84481dd3233eba4cdb608bc7a885fd8ca418c0806
Successfully built pyngrok
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
#Speech Recognition
!pip install pytube
!pip install git+https://github.com/openai/whisper.git -q
!pip install langdetect
#Translation, Summarization and Text Generation
!pip install evaluate
!pip install transformers sentencepiece datasets
!pip install rouge_score#No es necesaria
!pip install nltk
#Deploy
!pip install flask_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 KB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 108.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 KB 16.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=edf277dfd7854321e16612383a98cd11f5afce7a20015e7dd8a5ef92fcf47669
  Stored in directory: /root/.cache/pip/wheels/13/c7/b0/79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect
Looking in indexes: htt

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, render_template, request

#Speech Recognition
import pytube
import whisper
#Language Detection
from langdetect import detect
#Tranformers
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import pipeline

In [ ]:
app = Flask(__name__)
run_with_ngrok(app)

def get_audio(url):
    data = pytube.YouTube(url)
    # Converting and downloading as 'MP4' file
    audio = data.streams.get_audio_only()
    path = audio.download()#Get file from return of the method
    #Get Lyrics
    model = whisper.load_model('large')#large
    text = model.transcribe(path)
    print(text)
    text_output = text['text'].replace('♪','').strip()
    print(text_output)
    return text_output

activate_translation = False
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_repo = 't5-base'
LANG_TOKEN_MAPPING = {
    'en': '<en>',
    'es': '<es>'
}
model_trans = 'google/mt5-small'
tokenizer = AutoTokenizer.from_pretrained(model_trans)
model_t = AutoModelForSeq2SeqLM.from_pretrained('/content/drive/MyDrive/translation')
#model_t = model_t.cuda()



def language_dectection(text_output):
    global activate_translation
    detection = detect(text_output)
    print(detection)#Debug
    if detection != 'en':
        activate_translation = True
    else:
        activate_translation = False
    return detection


def summarization(text_output):
    # load local model
    if activate_translation == True and language_dectection(text_output) == 'es':
        model = (AutoModelForSeq2SeqLM
         .from_pretrained("/content/drive/MyDrive/Flask_model/trained_for_summarization_es")
         )
        seq2seq = pipeline("summarization", model=model, tokenizer='google/mt5-base')
        result = seq2seq(text_output, min_length=20, max_length=100)
        return result

    else:
        summarizer = pipeline("summarization", model=model_repo, tokenizer=model_repo, framework="tf")
        summary_text = summarizer(text_output, max_length=100, min_length=5, do_sample=False)[0]['summary_text']
        return summary_text



def encode_str(text, tokenizer, seq_len):  
#Tokenize,pad to max length and encode to ids Returns tensor with tocken ids
  input_ids = tokenizer.encode(
  text=text,
  return_tensors = 'pt',
  padding = 'max_length',
  truncation = True,
  max_length = seq_len)
  return input_ids[0]

def translation(summary):
    input_text = summary
    language = '<en>'
    print(language,input_text)
    input_ids = encode_str(
        text = language+input_text,
        tokenizer = tokenizer,
        seq_len = model_t.config.max_length
        )
    input_ids = input_ids.unsqueeze(0)
    print(input_ids)
    output_tokens = model_t.generate(input_ids,num_beams=10, num_return_sequences=1, length_penalty = 1, no_repeat_ngram_size=2)
    
    TRANS=tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    return TRANS

@app.route('/')
def home():
    return render_template("home.html")

#@app.route('/', methods = ['POST'])
#def load():
    #return render_template("home2.html")

@app.route("/", methods = ["POST","GET"])
def predict():
# taking the input
  text = request.form['text']
# preprocessing the text
  audio = get_audio(text)
  language_dectection(audio)
  summary = summarization(audio)
  print(activate_translation)
  print(summary)
  if activate_translation:
    translate = translation(summary[0]['summary_text'])
  else:
    translate = summary
  print(translate)
  return render_template("home.html", pred=" Summary: {}".format(translate))

if __name__ == '__main__':
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://8528-34-139-24-1.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [30/Jan/2023 14:55:55] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Jan/2023 14:55:55] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [30/Jan/2023 14:55:55] "GET /favicon.ico HTTP/1.1" 404 -


{'text': ' Ha sido el tema que ha acaparado la atención política en Washington, el hallazgo de documentos clasificados en las residencias de la expresidente Donald Trump y el presidente Joe Biden. Un tema que según la encuesta revelada este domingo por NBC, también está en la mira de los estadounidenses. Un 67% de los encuestados dijo sentirse preocupado por los hallazgos, a pesar de la diferencia en cómo respondieron ambos a esta controversia. Al secretario de justicia nombrar un fiscal especial para ambos, Biden y Trump, se crea una falsa equivalencia en la percepción del público. En otro tema, un 55% dice estar preocupado en que la nueva Cámara de Representantes, controlada por los republicanos, pase más tiempo investigando a Biden y no resolviendo otras prioridades. No puede ser simplemente prioridad, investigación en los próximos dos años, ni siquiera dos años, año y largo que queda de este Congreso 118, porque entonces ya la percepción del votante en mi opinión en el 2024 sería d

...

"The political attention in America is in the sights of the Americans.
{'text': ' Ha sido el tema que ha acaparado la atención política en Washington, el hallazgo de documentos clasificados en las residencias de la expresidente Donald Trump y el presidente Joe Biden. Un tema que según la encuesta revelada este domingo por NBC, también está en la mira de los estadounidenses. Un 67% de los encuestados dijo sentirse preocupado por los hallazgos, a pesar de la diferencia en cómo respondieron ambos a esta controversia. Al secretario de justicia nombrar un fiscal especial para ambos, Biden y Trump, se crea una falsa equivalencia en la percepción del público. En otro tema, un 55% dice estar preocupado en que la nueva Cámara de Representantes, controlada por los republicanos, pase más tiempo investigando a Biden y no resolviendo otras prioridades. No puede ser simplemente prioridad, investigación en los próximos dos años, ni siquiera dos años, año y largo que queda de este Congreso 118, porque

/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


True
[{'summary_text': 'La atención política en Estados Unidos está en la mira de los estadounidenses.'}]
<en> La atención política en Estados Unidos está en la mira de los estadounidenses.
tensor([[  1042,    278,    669,   2470,    259,    262,  20230,  11982,    289,
            259,  16710,  16854,   1957,    289,    283,  18355,    269,    595,
           9136, 209497,  47938,    260,      1,      0,      0,      0,      0,
              0,      0,      0,      0,      0,      0,      0,      0,      0,
              0,      0,      0,      0]])


/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 40 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
INFO:werkzeug:127.0.0.1 - - [30/Jan/2023 15:01:22] "POST / HTTP/1.1" 200 -


"The political attention in America is in the sights of the Americans.
